In [ ]:
import pickle
import random

import numpy as np
import torch
import torch.nn as nn

from adaptive_softmax.adaptive_softmax import AdaptiveLoss
import adaptive_softmax.model as Model
from utils.corpus import Corpus
import utils.dictionary as dictionary


def list2word(dictionary, l):
    words = []
    for i in l:
        words.append(dictionary[int(i)])
    return words

## Test language model

In [ ]:
checkpoint = './model/rocstoryline_data/aslm_new_model_v3.pt'
with open(checkpoint, 'rb') as f:
    model = torch.load(f)
model.eval()

device = torch.device("cpu")
model = model.to(device)

In [ ]:
data = './data/rocstoryline_data/disc_data'
dic = './data/rocstoryline_data/all-vocab.pkl'

In [ ]:
corpus = Corpus(data, dic)
ntokens = len(corpus.dictionary.idx2word)
cutoffs = [4000, 20000] + [ntokens]
EOT_id = corpus.dictionary.word2idx['<EOT>']
EOS_id = corpus.dictionary.word2idx['<EOS>']
EOL_id = corpus.dictionary.word2idx['<EOL>']
sent_id = corpus.dictionary.word2idx['</s>']
print('Total words        :{:>10d}'.format(len(corpus.dictionary.idx2word)))
print('EOT id             :{}\n'
      'EOL id             :{}\n'
      'EOS id             :{}\n'
      '</s> id            :{}'.format(EOT_id, EOL_id, EOS_id, sent_id))

In [ ]:
test_tokens = []
with open(corpus.splits['train'], 'r') as f:
    for line in f:
        words = line.split() + ['<EOS>']
        test_tokens += map(corpus.dictionary.__getitem__, words)
test_tokens = np.array(test_tokens)
test_stories = []
eos_pos = (test_tokens == eos_id).nonzero()[0]
test_stories.append(test_tokens[:eos_pos[0]+1])
for i in range(len(eos_pos)-1):
    test_stories.append(test_tokens[eos_pos[i]+1:eos_pos[i+1]+1])
print('Num of test: {}'.format(len(test_stories)))

In [ ]:
outf = './output/rocstory_data/out.txt'
nsents = 5
nwords = 100
temperature = 1.0

In [ ]:
model.full = True
hidden = model.init_hidden(1)
input = torch.rand(1, 1).mul(ntokens).long()
input = input.to(device)

tokens = test_tokens.copy()
# with open(outf, 'w') as outf:
for nsent in range(nsents):
    print('{:-^80}'.format(nsent))
    story = test_stories[nsent]
    cond_length = (story==EOL_id).nonzero()[0][0]+2
    print('Condition prefix: ', list2word(corpus.dictionary, story[:cond_length]))
#     hidden = model.init_hidden(1)
    for i in range(cond_length):
        word_idx = int(story[i])
        input.data.fill_(word_idx)
        output, hidden = model(input, hidden)
    for i in range(nwords):
        word_weights = output.squeeze().data.div(temperature).exp()
        word_idx = int(torch.multinomial(word_weights, 1)[0])
#         word_idx = int(torch.argmax(word_weights))
        word = corpus.dictionary.idx2word[word_idx]
        print(word, end=' ')
        input.data.fill_(word_idx)
        output, hidden = model(input, hidden)
        if word_idx == eos_id:
            print('')
            break

## Reuse entailment model

First compare their vocab with our vocab, and try to merge them.

In [ ]:
their_vocab_path = '/mnt/ssd/l2w/models/tbooks/vocab-converted.pickle'
our_vocab_path = '/mnt/ssd/nlp_project/project2/data/rocstoryline_data/all-vocab.pkl'

with open(their_vocab_path, 'rb') as f:
    their_dic = pickle.load(f)
our_dic = corpus.dictionary

In [ ]:
their_dic


## Evaluate mix model

Run the full mix model to get what scores are given to the candidate sentences, in order
to demonstrate.

raw scores are: lexical, relevance, repetition.

In [ ]:
%load_ext autoreload
%autoreload 2

In [55]:
import sys, argparse, pickle, os, random
from importlib import import_module
import torch
import numpy as np

from decoder import predictors, decoders
import adaptive_softmax.model

parser = argparse.ArgumentParser()
parser.add_argument('--data', type=str, default='input.txt',
                    help='text file containing initial strings to continue')
parser.add_argument('--skip', type=int, default=0,
                    help='number of lines to skip in data file before beginning')
parser.add_argument('--out', type=str, default='/tmp/l2w_output.txt',
                    help='text file to write generations to')
parser.add_argument('--lm', type=str, default='lm.pt',
                    help='lm to use for decoding')
parser.add_argument('--dic', type=str, default='dic.pickle',
                    help='dic to use for lm')
parser.add_argument('--print', action='store_true',
                    help='whether to print output to stdout (in addition to writing it to a file)')
parser.add_argument('--both', action='store_true',
                    help='also include pure LM output')
parser.add_argument('--gen_disc_data', action='store_true',
                    help='generator discriminator data from LM')
parser.add_argument('--epochs', type=int, default=1,
                    help='how many times to go through the input file')
parser.add_argument('--verbosity', type=int, default=0,
                    help='how verbose to be during decoding')
parser.add_argument('--cuda', action='store_true',
                    help='whether use CUDA')
parser.add_argument('--gpu', type=int,  default=0,
                    help='which gpu to use')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed')
parser.add_argument('--trip', action='store_true',
                    help='random seed')
## Learning
parser.add_argument('--learn', action='store_true')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--save_every', type=int, default=1)
## Decoding Stuff
parser.add_argument('--beam_size', type=int, default=10,
                    help='number of candidates in beam at every step')
parser.add_argument('--term', type=str, default='<end>',
                    help='what string to use as the end token')
parser.add_argument('--sep', type=str, default='</s>',
                    help='what string to use as the sentence seperator token')
parser.add_argument('--temp', type=float, default=None,
                    help='temperature, if using stochastic decoding')
parser.add_argument('--ranking_loss', action='store_true',
                    help='metaweight learning ranking loss')
parser.add_argument('--paragraph_level_score', action='store_true',
                    help='paragraph level score')
# Arbitrary Scorers
parser.add_argument('--scorers', type=str, default=None,
                    help='tsv with scorer information')
args = parser.parse_args([
    '--cuda',
    '--data', './data/new_roc_data_v3/weight_data/p1lm_new_roc_data_v3/test.tsv',
    '--lm', './model/new_roc_data_v3/p1lm_new_roc_data_v3.pt',
    '--dic', './data/new_roc_data_v3/all-vocab.pkl', 
    '--scorers', './model/new_roc_data_v3/scorer_weight_p1lm_new_roc_data_v3_norankloss.tsv',
    '--print',
    '--verbosity', '1',
    '--beam_size', '10'
#     '--temp', '0.8'
])
# args = parser.parse_args([
#     '--cuda',
#     '--data', './data/new_roc_data/weight_data/project1_model_new_roc_data/storylines_50.txt.processed',
#     '--lm', './model/new_roc_data/project1_model_new_roc_data_e100.pt',
#     '--dic', './data/new_roc_data/all-vocab.pkl', 
#     '--scorers', './model/new_roc_data/scorer_weight_project1_model_new_roc_data.tsv',
#     '--print',
#     '--verbosity', '1',
#     '--beam_size', '10'
# #     '--temp', '0.8'
# ])

np.random.seed(args.seed)

torch.manual_seed(args.seed)
# create device agnostic device
if torch.cuda.is_available():
    if not args.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")
    else:
        torch.cuda.manual_seed(args.seed)
        device = torch.device("cuda", args.gpu)
else:
    device = torch.device("cpu")

print("Load model")
with open(args.lm, 'rb') as model_file:
    model = torch.load(model_file, map_location='cuda:0')

model.eval()
with open(args.dic, 'rb') as dic_file:
    dictionary = pickle.load(dic_file)
predictor = predictors.RNNPredictor(model, len(dictionary), asm=True,
    device=device)

print("Creating scorers and decoder")
scorer_config, scorers, coefs = [], [], []
if args.scorers:
    with open(args.scorers) as scorer_file:
        for line in scorer_file:
            fields = line.strip().split('\t')
            scorer_config.append(fields)
            weight, path, classname  = fields[:3]
            weight = float(weight)
            arg_scripts = fields[3]
            module = import_module(path)
            constructor = getattr(module, classname) 
            scorer =  constructor(arg_scripts)
            scorers.append(scorer)
            coefs.append(weight)
decoder = decoders.BeamRerankDecoder(predictor,
                                     scorers,
                                     coefs,
                                     learn=args.learn,
                                     lr=args.lr,
                                     ranking_loss=args.ranking_loss,
                                     beam_size=args.beam_size,
                                     temperature=args.temp,
                                     terms=[dictionary['</s>']],
                                     forbidden=[dictionary['<unk>'], dictionary['<EOT>'], dictionary['<EOL>'],dictionary['<EOS>']],
                                     sep=dictionary[args.sep],
                                     verbosity=args.verbosity,
                                     dictionary=dictionary)
lm_decoder = decoders.BeamRerankDecoder(predictor,
                                     [],
                                     [],
                                     [],
                                     [],
                                     beam_size=args.beam_size,
                                     temperature=args.temp,
                                     terms=[dictionary['</s>']],
                                     forbidden=[dictionary['<unk>'], dictionary['<EOT>'], dictionary['<EOL>'], dictionary['<EOS>']],
                                     sep=dictionary[args.sep],
                                     verbosity=args.verbosity,
                                     dictionary=dictionary)

print("Start decoding")
avg, a_n = None, 0
for i in range(args.epochs):
    with open(args.data) as data_file, open(args.out, 'w') as out_file:
        for i, line in enumerate(data_file):
            if i < args.skip:
                continue
            if i not in [1]:
                continue
            if args.gen_disc_data:
                init_tokens = line.strip().lower().split()
                init_tokens_ints = [dictionary[token] for token in init_tokens]
            else:
                initial, continuation = line.split('\t')[:2]
                init_tokens = initial.strip().lower().split()
                true_cont_tokens = continuation.strip().lower().split()
                true_cont_ints = [dictionary[token] for token in true_cont_tokens]
                init_tokens_ints = [dictionary[token] for token in init_tokens]
            if args.learn:
                diff = decoder.decode(init_tokens_ints,
                                      true_cont_ints)
                out_str = '%f\n' % diff
            elif args.gen_disc_data:
                init = ' '.join(init_tokens)
                lm_pred_tokens_ints = lm_decoder.decode(init_tokens_ints, itos=dictionary)
                lm_pred_cont_tokens = [dictionary[token]
                                        for token in lm_pred_tokens_ints[len(init_tokens):]]
                lm_cont = ' '.join(lm_pred_cont_tokens)
                out_str = '%s\n' % lm_cont
            else:
                pred_tokens_ints = decoder.decode(init_tokens_ints, itos=dictionary)
                pred_cont_tokens = [dictionary[token]
                                for token in pred_tokens_ints[len(init_tokens):]]
                init = ' '.join(init_tokens)
                cont = ' '.join(pred_cont_tokens)
                if args.both:
                    lm_pred_tokens_ints = lm_decoder.decode(init_tokens_ints, itos=dictionary)
                    lm_pred_cont_tokens = [dictionary[token]
                                            for token in lm_pred_tokens_ints[len(init_tokens):]]
                    lm_cont = ' '.join(lm_pred_cont_tokens)
                    out_str = '%s***\t%s***\t%s\n' % (init, cont, lm_cont)
                else:
                    out_str = '%s\t%s\n' % (init, cont)
    
            out_file.write(out_str)
            out_file.flush()
            if args.print:
                print(out_str, end='')


Load model
Creating scorers and decoder
Start decoding
-------------------------------Begin current beam-------------------------------
i | lm score: -3.530677 | 
john | lm score: -3.781230 | 
tom | lm score: -3.841538 | 
tim | lm score: -3.872518 | 
charles | lm score: -4.370919 | 
sam | lm score: -4.425754 | 
bob | lm score: -4.530040 | 
chris | lm score: -4.543912 | 
jim | lm score: -4.578883 | 
mike | lm score: -4.580812 | 
--------------------------------End current beam--------------------------------
------------------------------Begin expanded beam-------------------------------
i was | lm score: -5.338985 | 
i loved | lm score: -5.377156 | 
i had | lm score: -6.074633 | 
i liked | lm score: -6.225971 | 
i wanted | lm score: -6.322842 | 
i always | lm score: -6.590687 | 
i hated | lm score: -7.056694 | 
i read | lm score: -7.156725 | 
i went | lm score: -7.778331 | 
i decided | lm score: -7.903573 | 
john loved | lm score: -5.179552 | 
john was | lm score: -5.548147 | 
john lik

charles loved books . | lm score: -9.758338 | 
charles loved books in | lm score: -13.517861 | 
charles loved books and | lm score: -13.526202 | 
charles loved books with | lm score: -14.005401 | 
charles loved books for | lm score: -14.053238 | 
charles loved books at | lm score: -14.149851 | 
charles loved books to | lm score: -14.429704 | 
charles loved books on | lm score: -14.437751 | 
charles loved books of | lm score: -14.440667 | 
charles loved books from | lm score: -14.451047 | 
-------------------------------End expanded beam--------------------------------
-------------------------------Begin current beam-------------------------------
charles loved reading . | lm score: -10.776195 | lex,rel,rep score:  [-0.18740584 -0.31343052 -0.45575115] | adjusted score: -18.111435 | 
charles loved reading . | lm score: -10.776195 | lex,rel,rep score:  [-0.18740584 -0.31343052 -0.45575115] | adjusted score: -18.111435 | 
sam hated reading . | lm score: -11.499934 | lex,rel,rep score:  [

charles loved reading books . </s> every | lm score: -15.231171 | 
charles loved reading books . </s> he | lm score: -10.433249 | 
charles loved reading books . </s> his | lm score: -12.866037 | 
charles loved reading books . </s> one | lm score: -12.906037 | 
charles loved reading books . </s> but | lm score: -12.951144 | 
charles loved reading books . </s> so | lm score: -14.107219 | 
charles loved reading books . </s> she | lm score: -14.210841 | 
charles loved reading books . </s> the | lm score: -14.421068 | 
charles loved reading books . </s> when | lm score: -14.696796 | 
charles loved reading books . </s> it | lm score: -14.796158 | 
charles loved reading books . </s> every | lm score: -15.231171 | 
charles loved reading books . </s> he | lm score: -10.433249 | 
charles loved reading books . </s> his | lm score: -12.866037 | 
charles loved reading books . </s> one | lm score: -12.906037 | 
charles loved reading books . </s> but | lm score: -12.951144 | 
charles loved reading bo

charles loved reading books . </s> he wanted | lm score: -13.707042 | 
charles loved reading books . </s> he thought | lm score: -13.712113 | 
charles loved reading books . </s> he always | lm score: -13.924850 | 
charles loved reading books . </s> he liked | lm score: -14.038939 | 
charles loved reading books . </s> he did | lm score: -14.053344 | 
-------------------------------End expanded beam--------------------------------
-------------------------------Begin current beam-------------------------------
charles loved reading books . </s> but he | lm score: -13.320325 | lex,rel,rep score:  [-0.27993518 -0.14063768 -0.55286   ] | adjusted score: -19.731018 | 
charles loved reading books . </s> he thought | lm score: -13.712113 | lex,rel,rep score:  [-0.26743117 -0.13440308 -0.4976782 ] | adjusted score: -19.682575 | 
charles loved reading books . </s> his books | lm score: -16.534279 | lex,rel,rep score:  [-0.19460449 -0.11145183 -0.6564983 ] | adjusted score: -22.532475 | 
charles 

-------------------------------Begin current beam-------------------------------
charles loved reading books . </s> he always enjoyed | lm score: -17.594231 | lex,rel,rep score:  [-0.24929594 -0.15269905 -0.4113562 ] | adjusted score: -23.219702 | 
charles loved reading books . </s> he thought to | lm score: -17.178890 | lex,rel,rep score:  [-0.43150344 -0.14083703 -0.4511011 ] | adjusted score: -24.124748 | 
charles loved reading books . </s> his books was | lm score: -18.387405 | lex,rel,rep score:  [-0.60231304 -0.11479049 -0.67465806] | adjusted score: -27.337194 | 
charles loved reading books . </s> he always wanted | lm score: -16.007992 | lex,rel,rep score:  [-0.2943776  -0.15429819 -0.45707193] | adjusted score: -22.195892 | 
charles loved reading books . </s> he thought his | lm score: -16.759706 | lex,rel,rep score:  [-0.25025576 -0.13709502 -0.4945627 ] | adjusted score: -22.627630 | 
charles loved reading books . </s> he decided to | lm score: -13.097968 | lex,rel,rep score

...

charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income that his computer was a great ! <EOS> | lm score: -103.178337 | 
charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income that his computer was a great ! of | lm score: -103.530792 | 
charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income that his computer was a great ! to | lm score: -104.017685 | 
charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income that his computer was a great ! in | lm score: -104.454742 | 
c

charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income that his computer was a great ! </s> </s> | lm score: -103.905533 | 
charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income that his computer was a great ! </s> . | lm score: -104.531418 | 
charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income that his computer was a great ! </s> the | lm score: -104.622002 | 
charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income that his computer was a great ! </s> now | lm sco

charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income for writing on the language . </s> . </s> after | lm score: -106.439728 | 
charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income that his computer was a lot . </s> now the | lm score: -104.645432 | 
charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income that his computer was a lot . </s> now a | lm score: -106.981346 | 
charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income that his computer was a lot . </s> now 

-------------------------------Begin current beam-------------------------------
charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income that his computer was a great time . </s> after the | lm score: -107.136261 | lex,rel,rep score:  [-0.31455985 -0.08613606 -8.105791  ] | adjusted score: -151.400848 | 
charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income that his computer was a great time . </s> the book | lm score: -108.854774 | lex,rel,rep score:  [-0.34832594 -0.08265859 -8.127713  ] | adjusted score: -153.422562 | 
charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a lot of income that his computer was

literature vs math <eot> </s> loved literature path indecisive more books love strong write problems <eol> </s>	charles loved reading books . </s> he decided to write an essay about a language . </s> one day , he wrote a book online . </s> he had a lot of research . </s> now he has a great time ! </s>


# Evaluate sub model

test the model prediction of sub model

In [ ]:
import argparse, pickle, os, sys
import numpy as np
from collections import Counter
from collections import defaultdict
from itertools import chain

from scipy.special import expit
import torch
from torchtext import data
from torchtext import vocab
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd

# path = os.path.realpath(__file__)
# path = path[:path.rindex('/')+1]
# sys.path.insert(0, os.path.join(path, '../utils/'))
# sys.path.insert(0, os.path.join(path, '../word_rep/'))

from trainers.cnn_context_classifier import CNNContextClassifier
from trainers.pool_ending_classifier import PoolEndingClassifier
from trainers.reprnn import RepRNN

parser = argparse.ArgumentParser()
# Data
parser.add_argument('data_dir', type=str, help='path to data directory')
parser.add_argument('--save_to', type=str, default='', help='path to save model to')
parser.add_argument('--load_model', type=str, default='', help='existing model file to load')
parser.add_argument('--dic', type=str, default='dic.pickle',
                    help='lm dic to use as vocabulary')
# Model
parser.add_argument('--decider_type', type=str, default='cnncontext',
                    help='Decider classifier type [cnncontext, poolending, reprnn]')
# Run Parameters
parser.add_argument('--batch_size',
                    type=int,
                    default=64,
                    help='number of examples to process in parallel')
parser.add_argument('--num_epochs',
                    type=int,
                    default=5,
                    help='number of times to run through training set')
parser.add_argument('--stop_threshold',
                    type=float,
                    default=0.99,
                    help='Early stopping threshold on validation accuracy')
parser.add_argument('--lr',
                    type=float,
                    default=0.001,
                    help='learning rate for optimizer')
parser.add_argument('--adam',
                    action='store_true',
                    help='train with adam optimizer')
parser.add_argument('--train_prefixes',
                    action='store_true',
                    help='train on all ending prefixes')
parser.add_argument('--valid_only',
                    action='store_true',
                    help='use only validation set')
parser.add_argument('--ranking_loss',
                    action='store_true',
                    help='train based on ranking loss')
parser.add_argument('--margin_ranking_loss',
                    action='store_true',
                    help='train based on margin ranking loss')
# Model Parameters
parser.add_argument('--embedding_dim',
                    type=int,
                    default=300,
                    help='length of word embedding vectors')
parser.add_argument('--hidden_dim',
                    type=int,
                    default=300,
                    help='length of hidden state vectors')
parser.add_argument('--filter_size',
                    type=int,
                    default=3,
                    help='convolutional filter size')
parser.add_argument('--dropout_rate',
                    type=float,
                    default=0.5,
                    help='dropout rate')
parser.add_argument('--fix_embeddings',
                    action='store_true',
                    help='fix word embeddings')
# Output Parameters
parser.add_argument('--valid_every',
                    type=int,
                    default=128,
                    help='batch interval for running validation')
parser.add_argument('-p',
                    action='store_true',
                    help='use this flag to print samples of the data')
# args = parser.parse_args([
#     './data/new_roc_data_v3/rep_data/p1lm_new_roc_data_v3/',
#     '--load_model', './model/new_roc_data_v3/repetition_model_p1lm_new_roc_data_v3_norank_loss.pt',
#     '--dic', './data/new_roc_data_v3/all-vocab.pkl',
#     '--fix_embeddings', '--adam', '--train_prefixes',
#     '--decider_type', 'reprnn',
#     '--batch_size', '10',
# ])
args = parser.parse_args([
    './data/new_roc_data_v3/rel_data/p1lm_new_roc_data_v3/',
    '--load_model', './model/new_roc_data_v3/relevance_model_p1lm_new_roc_data_v3_norank_loss.pt',
    '--dic', './data/new_roc_data_v3/all-vocab.pkl',
    '--fix_embeddings', '--adam', '--train_prefixes',
    '--decider_type', 'reprnn',
    '--batch_size', '30',
])
# args = parser.parse_args([
#     './data/new_roc_data_v3/lex_data/p1lm_new_roc_data_v3/',
#     '--load_model', './model/new_roc_data_v3/lexical_model_p1lm_new_roc_data_v3_norank_loss.pt',
#     '--dic', './data/new_roc_data_v3/all-vocab.pkl',
#     '--fix_embeddings', '--adam', '--train_prefixes',
#     '--decider_type', 'reprnn',
#     '--batch_size', '10',
# ])

TEXT = data.Field(sequential=True, lower=True, include_lengths=True)

LABEL = data.Field(sequential=False, use_vocab=False, tensor_type=torch.FloatTensor, postprocessing=data.Pipeline(lambda x, y: float(x)))

if args.valid_only:
    train_name = 'valid.tsv'
else:
    train_name = 'disc_train.tsv'

print('Reading the data')
train, valid = data.TabularDataset.splits(
    path=args.data_dir,
    train=train_name, validation='valid.tsv',
    format='tsv',
    fields=[
        ('context', TEXT),
        ('generated', TEXT),
        ('gold', TEXT),
        ])

#TEXT.build_vocab(train)
# Read in the LM dictionary.
print('Building the dictionary')
with open(args.dic, 'rb') as dic_file:
    dictionary = pickle.load(dic_file)

# Reconstruct the dictionary in torchtext.
counter = Counter({'<unk>': 0, '</s>':0})
TEXT.vocab = vocab.Vocab(counter, specials=['<unk>', '</s>'])
TEXT.vocab.itos = dictionary.idx2word
TEXT.vocab.stoi = defaultdict(vocab._default_unk_index, dictionary.word2idx)

TEXT.vocab.load_vectors('glove.6B.%dd' % args.embedding_dim)
itos = TEXT.vocab.itos if args.p else None
print('Vocab size %d' % len(TEXT.vocab))

train_iter = data.Iterator(dataset=train, batch_size=args.batch_size,
        sort_key=lambda x: len(x.context), sort=True, repeat=False)
valid_iter = data.Iterator(dataset=valid, batch_size=args.batch_size, sort_key=lambda x: len(x.context), sort=True, repeat=False)

print('Initializing the model')

if args.load_model != '':
    print('Loading existing model!')
    with open(args.load_model, 'rb') as f:
        model = torch.load(f).cuda()
elif args.decider_type == 'cnncontext':
    model = CNNContextClassifier(len(TEXT.vocab), args.embedding_dim,
            args.hidden_dim, args.filter_size, args.dropout_rate,
            embed_mat=TEXT.vocab.vectors,
            fix_embeddings=args.fix_embeddings).cuda()
elif args.decider_type == 'poolending':
    model = PoolEndingClassifier(len(TEXT.vocab), args.embedding_dim,
            args.hidden_dim,
            embed_mat=TEXT.vocab.vectors,
            fix_embeddings=args.fix_embeddings).cuda()
elif args.decider_type == 'reprnn':
    model = RepRNN(len(TEXT.vocab), args.embedding_dim,
            args.hidden_dim,
            embed_mat=TEXT.vocab.vectors).cuda()
else:
  assert False, 'Invalid model type.'

loss_function = nn.BCEWithLogitsLoss()
margin_loss_function = nn.MarginRankingLoss()

parameters = filter(lambda p: p.requires_grad, model.parameters())
if args.adam:
    optimizer = optim.Adam(parameters, lr=args.lr)
else:
    optimizer = optim.SGD(parameters, lr=args.lr)

print('Evaluating model')
if args.load_model != '':
    model.eval()
    valid_iter.init_epoch()
    v_correct, v_total = 0, 0
    ones = 0
    for k, batch in enumerate(train_iter):
        #if k % 100 == 0:
        #    print(k)
        batch_size = batch.context[0].size()[1]

        decision_negative = model(batch.context[0],
            batch.generated, itos=itos)
        decision_positive = model(batch.context[0],
            batch.gold, itos=itos)

        if args.ranking_loss or args.margin_ranking_loss:
            decision = decision_positive - decision_negative
        else:
            # Evaluate predictions on gold
            decision = decision_positive

        decis = decision.data.cpu().numpy()
        predicts = (expit(decis))
        print(predicts)
        decis_neg = decision_negative.data.cpu().numpy()
        predicts_neg = (expit(decis_neg))
        print(predicts_neg)
        break
        v_correct += np.sum(np.equal(predicts, np.ones(batch_size)))
        v_total += batch_size
        ones += np.sum(predicts)
    print('Valid: %f' % (v_correct / v_total))
    print('%d ones %d zeros' % (ones, v_total - ones))


# early_stop = False
# for epoch in range(args.num_epochs):
#     if early_stop:
#         break
#     print('{:-^80}'.format('Epoch %d' % epoch))
#     train_iter.init_epoch()
#     correct, total = 0, 0
#     total_loss = 0
#     for b, batch in enumerate(train_iter):
#         model.train()
#         model.zero_grad()
#         batch_size = batch.context[0].size()[1]

#         def compute_loss(context, generated, gold):
#             decision_negative = model(context, generated, itos=itos)
#             decision_positive = model(context, gold, itos=itos)
#             if args.ranking_loss or args.margin_ranking_loss:
#                 decision = decision_positive - decision_negative
#             else:
#                 decision = decision_positive

#             x_loss = None
#             if args.ranking_loss:
#                 x_loss = loss_function(
#                   decision_positive - decision_negative,
#                   autograd.Variable(torch.ones(batch_size)).cuda())
#             elif args.margin_ranking_loss:
#                 # 1: positive ranked higher than negative
#                 x_loss = margin_loss_function(
#                   decision_positive, decision_negative,
#                   autograd.Variable(torch.ones(batch_size)).cuda())
#             else:
#                 x_loss = loss_function(decision_positive,
#                   autograd.Variable(torch.ones(batch_size)).cuda())
#                 x_loss += loss_function(decision_negative,
#                   autograd.Variable(torch.zeros(batch_size)).cuda())

#             return x_loss, decision

#         loss = None
#         if args.train_prefixes:
#             end_seq_len = max(batch.generated[0].size()[0],
#                               batch.gold[0].size()[0])
#             loss = 0
#             #length_range = chain(range(min(10, end_seq_len)), 
#             #                     range(10, end_seq_len, 5))
#             length_range = chain(range(0, min(10, end_seq_len-1), 2), 
#                                  range(10, min(end_seq_len, 30), 5),
#                                  iter([end_seq_len-1]))

#             for i in length_range:
#                 gen_len = min(i+1, batch.generated[0].size()[0])
#                 gold_len = min(i+1, batch.gold[0].size()[0])
#                 prefix_loss, decision = compute_loss(batch.context[0],
#                     (batch.generated[0][:gen_len,:].view(gen_len, -1),
#                     autograd.Variable(torch.ones(batch_size)*i).cuda()),
#                     (batch.gold[0][:gold_len,:].view(gold_len, -1),
#                     autograd.Variable(torch.ones(batch_size)*i).cuda()))
#                 loss += prefix_loss
#         else:
#             loss, decision = compute_loss(batch.context[0],
#                     batch.generated, batch.gold)

#         loss.backward()
#         total_loss += loss.data.item()
#         optimizer.step()

#         correct += np.sum(np.equal(
#             np.round(expit(decision.data.cpu().numpy())),
#             np.ones(batch_size)))
#         total += batch_size

#         if b % args.valid_every == 0:
#             model.eval()
#             valid_iter.init_epoch()
#             v_correct, v_total = 0, 0
#             ones = 0
#             for k, batch in enumerate(valid_iter):
#                 #if k % 100 == 0:
#                 #    print(k)
#                 batch_size = batch.context[0].size()[1]

#                 decision_negative = model(batch.context[0],
#                     batch.generated, itos=itos)
#                 decision_positive = model(batch.context[0],
#                     batch.gold, itos=itos)

#                 if args.ranking_loss or args.margin_ranking_loss:
#                     decision = decision_positive - decision_negative
#                 else:
#                     # Evaluate predictions on gold
#                     decision = decision_positive

#                 decis = decision.data.cpu().numpy()
#                 predicts = np.round(expit(decis))
#                 v_correct += np.sum(np.equal(predicts, np.ones(batch_size)))
#                 v_total += batch_size
#                 ones += np.sum(predicts)
#             acc = v_correct / v_total
#             print('{:^15}Batch: {:5d} Valid Acc:{:.2f} ({:5d} 1s {:5d} 0s)'
#                   .format('', b, acc,int(ones), int(v_total - ones)))
#             if v_correct / v_total > args.stop_threshold: # early stopping
#                 early_stop = True
#                 break

#     print('{:^10}Train Acc: {:.2f} Loss:{:5.4e}'
#         .format('', correct / total, total_loss / total))
#     # print('Saving model')
#     with open(args.save_to, 'wb') as f:
#         torch.save(model, f)



In [ ]:
m = nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)

In [ ]:
output.shape

In [ ]:
[TEXT.vocab.itos[i] for i in batch.context[0][0].cpu().numpy()]